# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 12 2023 11:41AM,264566,15,PNC459124A,"Person & Covey, Inc.",Released
1,Jun 12 2023 11:41AM,264566,15,PNC459136A,"Person & Covey, Inc.",Released
2,Jun 12 2023 11:41AM,264566,15,PNC459145A,"Person & Covey, Inc.",Released
3,Jun 12 2023 11:41AM,264566,15,PNC459148A,"Person & Covey, Inc.",Released
4,Jun 12 2023 11:40AM,264565,10,Med-9565829,MedStone Pharma LLC,Released
5,Jun 12 2023 11:40AM,264565,10,Med-9565830,MedStone Pharma LLC,Released
6,Jun 12 2023 11:38AM,264562,19,ELI06132023,Eli Lilly and Company,Released
7,Jun 12 2023 11:35AM,264561,10,9565693,MedStone Pharma LLC,Released
8,Jun 12 2023 11:35AM,264561,10,9565695,MedStone Pharma LLC,Released
9,Jun 12 2023 11:35AM,264561,10,9565696,MedStone Pharma LLC,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
17,264558,Released,40
18,264561,Released,8
19,264562,Released,1
20,264565,Released,2
21,264566,Released,4


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
264558,NaN,40.0
264561,NaN,8.0
264562,NaN,1.0
264565,NaN,2.0
264566,NaN,4.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
264534,0.0,43.0
264535,0.0,1.0
264537,0.0,1.0
264540,9.0,3.0
264543,0.0,2.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
264534,0,43
264535,0,1
264537,0,1
264540,9,3
264543,0,2


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,264534,0,43
1,264535,0,1
2,264537,0,1
3,264540,9,3
4,264543,0,2


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,264534,,43
1,264535,,1
2,264537,,1
3,264540,9,3
4,264543,,2


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 12 2023 11:41AM,264566,15,"Person & Covey, Inc."
4,Jun 12 2023 11:40AM,264565,10,MedStone Pharma LLC
6,Jun 12 2023 11:38AM,264562,19,Eli Lilly and Company
7,Jun 12 2023 11:35AM,264561,10,MedStone Pharma LLC
15,Jun 12 2023 11:30AM,264550,10,Eywa Pharma Inc.
18,Jun 12 2023 11:20AM,264558,15,"Brookfield Pharmaceuticals, LLC"
58,Jun 12 2023 11:01AM,264557,10,Eywa Pharma Inc.
59,Jun 12 2023 10:57AM,264556,15,Lonza
60,Jun 12 2023 10:53AM,264555,19,"NAPP Technologies, LLC"
61,Jun 12 2023 10:52AM,264554,22,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Jun 12 2023 11:41AM,264566,15,"Person & Covey, Inc.",,4
1,Jun 12 2023 11:40AM,264565,10,MedStone Pharma LLC,,2
2,Jun 12 2023 11:38AM,264562,19,Eli Lilly and Company,,1
3,Jun 12 2023 11:35AM,264561,10,MedStone Pharma LLC,,8
4,Jun 12 2023 11:30AM,264550,10,Eywa Pharma Inc.,,3
5,Jun 12 2023 11:20AM,264558,15,"Brookfield Pharmaceuticals, LLC",,40
6,Jun 12 2023 11:01AM,264557,10,Eywa Pharma Inc.,,1
7,Jun 12 2023 10:57AM,264556,15,Lonza,,1
8,Jun 12 2023 10:53AM,264555,19,"NAPP Technologies, LLC",,1
9,Jun 12 2023 10:52AM,264554,22,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 12 2023 11:41AM,264566,15,"Person & Covey, Inc.",4,
1,Jun 12 2023 11:40AM,264565,10,MedStone Pharma LLC,2,
2,Jun 12 2023 11:38AM,264562,19,Eli Lilly and Company,1,
3,Jun 12 2023 11:35AM,264561,10,MedStone Pharma LLC,8,
4,Jun 12 2023 11:30AM,264550,10,Eywa Pharma Inc.,3,
5,Jun 12 2023 11:20AM,264558,15,"Brookfield Pharmaceuticals, LLC",40,
6,Jun 12 2023 11:01AM,264557,10,Eywa Pharma Inc.,1,
7,Jun 12 2023 10:57AM,264556,15,Lonza,1,
8,Jun 12 2023 10:53AM,264555,19,"NAPP Technologies, LLC",1,
9,Jun 12 2023 10:52AM,264554,22,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 12 2023 11:41AM,264566,15,"Person & Covey, Inc.",4,
1,Jun 12 2023 11:40AM,264565,10,MedStone Pharma LLC,2,
2,Jun 12 2023 11:38AM,264562,19,Eli Lilly and Company,1,
3,Jun 12 2023 11:35AM,264561,10,MedStone Pharma LLC,8,
4,Jun 12 2023 11:30AM,264550,10,Eywa Pharma Inc.,3,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Jun 12 2023 11:41AM,264566,15,"Person & Covey, Inc.",4.0,NaN
1,Jun 12 2023 11:40AM,264565,10,MedStone Pharma LLC,2.0,NaN
2,Jun 12 2023 11:38AM,264562,19,Eli Lilly and Company,1.0,NaN
3,Jun 12 2023 11:35AM,264561,10,MedStone Pharma LLC,8.0,NaN
4,Jun 12 2023 11:30AM,264550,10,Eywa Pharma Inc.,3.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jun 12 2023 11:41AM,264566,15,"Person & Covey, Inc.",4.0,0.0
1,Jun 12 2023 11:40AM,264565,10,MedStone Pharma LLC,2.0,0.0
2,Jun 12 2023 11:38AM,264562,19,Eli Lilly and Company,1.0,0.0
3,Jun 12 2023 11:35AM,264561,10,MedStone Pharma LLC,8.0,0.0
4,Jun 12 2023 11:30AM,264550,10,Eywa Pharma Inc.,3.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2116391,76.0,0.0
15,1322779,66.0,11.0
18,264535,1.0,0.0
19,793657,5.0,9.0
21,529094,1.0,1.0
22,264554,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2116391,76.0,0.0
1,15,1322779,66.0,11.0
2,18,264535,1.0,0.0
3,19,793657,5.0,9.0
4,21,529094,1.0,1.0
5,22,264554,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,76.0,0.0
1,15,66.0,11.0
2,18,1.0,0.0
3,19,5.0,9.0
4,21,1.0,1.0
5,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,76.0
1,15,Released,66.0
2,18,Released,1.0
3,19,Released,5.0
4,21,Released,1.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,18,19,21,22
Status,,,,,,
Executing,0.0,11.0,0.0,9.0,1.0,0.0
Released,76.0,66.0,1.0,5.0,1.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,18,19,21,22
0,Executing,0.0,11.0,0.0,9.0,1.0,0.0
1,Released,76.0,66.0,1.0,5.0,1.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,18,19,21,22
0,Executing,0.0,11.0,0.0,9.0,1.0,0.0
1,Released,76.0,66.0,1.0,5.0,1.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()